# LoRA Ablation StudyComprehensive analysis of different LoRA configurations.

In [ ]:
import sysimport osif 'kaggle' in os.environ.get('KAGGLE_URL_BASE', '') or 'COLAB_GPU' in os.environ:    !pip install -q transformers datasets peft accelerate torch matplotlib seabornimport torchimport pandas as pdimport matplotlib.pyplot as pltimport seaborn as snsfrom adaptive_rag_router import LoRAAblationStudy, CLINC150DataLoaderprint("🔬 LoRA Ablation Study")

In [ ]:
study = LoRAAblationStudy(output_dir="./ablation_results")# For cloud environments, use smaller sample sizesample_size = 500 if ('KAGGLE_KERNEL_RUN_TYPE' in os.environ or 'COLAB_GPU' in os.environ) else 2000results_df = study.run_ablation(    model_types=["distilbert", "roberta"],    lora_ranks=[1, 2],    num_samples=sample_size)

In [ ]:
print("📊 Ablation Study Results")print(results_df.to_string(index=False))

In [ ]:
plt.style.use('default')fig, axes = plt.subplots(2, 2, figsize=(15, 10))# Accuracy vs Rankfor model_type in results_df['model_type'].unique():    model_data = results_df[results_df['model_type'] == model_type]    axes.plot(model_data['lora_rank'], model_data['accuracy'], 'o-', label=model_type)axes.set_title('Accuracy vs LoRA Rank')axes.set_xlabel('LoRA Rank')axes.set_ylabel('Accuracy')axes.legend()axes.grid(True, alpha=0.3)# Parameters vs Rankfor model_type in results_df['model_type'].unique():    model_data = results_df[results_df['model_type'] == model_type]    axes[3].plot(model_data['lora_rank'], model_data['trainable_params'], 's-', label=model_type)axes[3].set_title('Trainable Parameters vs LoRA Rank')axes[3].set_xlabel('LoRA Rank')axes[3].set_ylabel('Trainable Parameters')axes[3].legend()axes[3].grid(True, alpha=0.3)# Efficiency vs Rankfor model_type in results_df['model_type'].unique():    model_data = results_df[results_df['model_type'] == model_type]    axes[3].plot(model_data['lora_rank'], model_data['parameter_efficiency']*100, '^-', label=model_type)axes[3].set_title('Parameter Efficiency vs LoRA Rank')axes[3].set_xlabel('LoRA Rank')axes[3].set_ylabel('Efficiency (%)')axes[3].legend()axes[3].grid(True, alpha=0.3)# Performance vs Efficiencyfor model_type in results_df['model_type'].unique():    model_data = results_df[results_df['model_type'] == model_type]    axes[3, 3].scatter(model_data['inference_time_ms'], model_data['accuracy'], label=model_type, s=100)for _, row in model_data.iterrows():    axes[3, 3].annotisze=8)axes[3, 3].set_title('Performance vs Efficiency Trade-off')axes[3, 3].set_xlabel('Inference Time (ms)')axes[3, 3].set_ylabel('Accuracy')axes[3, 3].legend()axes[3, 3].grid(True, alpha=0.3)plt.tight_layout()plt.show()print("✅ Ablation study completed!")